In [1]:
import os
import json
import geopandas as gpd
import numpy as np
import pandas as pd
import shapely
from sqlalchemy import create_engine, event
from sqlite3 import dbapi2 as sqlite

Setup Paths

In [3]:
cwd = r'C:\Users\uscd675041\WSP O365\BART-TDLU - Documents'

networks = os.path.join(cwd, 'networks')
data = os.path.join(cwd, 'data')

Create connection to SQLite (Spatialite) Database

In [2]:
shared_lib = r'C:\Program Files\spatialite'
os.environ["PATH"] = shared_lib + ";" + os.environ["PATH"]

#if os.path.exists(os.path.join(networks, 'l21.sqlite')):
#    os.remove(os.path.join(networks, 'l21.sqlite'))

engine = create_engine(f'sqlite:///{os.path.join(networks, "l21.sqlite")}', module=sqlite)

Helper Methods

In [6]:
@event.listens_for(engine, 'connect')
def connect(dbapi_connection, connection_rec):
    dbapi_connection.enable_load_extension(True)
    dbapi_connection.execute('SELECT load_extension("mod_spatialite")')
    
def simplify(x):
    if isinstance(x, list):
        return ','.join([i for i in set(x) if i and i != 'nan'])
    if x in ['nan', '', np.nan]:
        return None
    return x

If the database is new, enable spatial feature.

In [7]:
# create spatialite metadata
print('creating spatial metadata...')
engine.execute("SELECT InitSpatialMetaData(1);")

creating spatial metadata...


Read in list of counties

In [8]:
counties = pd.read_csv(os.path.join(data, 'counties.csv'))


Write to Spatialite

In [13]:
count = 0
for sfips, cfips, county_name in zip(counties['STATE_FIPS'], counties['COUNTY_FIPS'], counties['COUNTY_NAME']):
    
    county_name = county_name.replace(' ', '')
    exists = False
    
    for ext in ['', f'_{county_name}']:
        file_name = os.path.join(networks, county_name, f'step8_links{ext}.geojson')
        if os.path.exists(file_name):
            exists = True
            break
    
    print(f'{county_name}:{exists}')
    
    if not exists:
        continue
    
    
    
    
    with open(file_name, 'r') as f:
        j = json.load(f)
    links = gpd.GeoDataFrame.from_features(j)
    links['geometry'] = links['geometry'].apply(lambda x: shapely.wkb.dumps(x))


    for col in ['roadClass', 'ref', 'name', 'service', 'lanes', 'maxspeed', 'roadway']:
        links[col] = links[col].apply(simplify)

    links['state_fips'] = sfips
    links['county_fips'] = cfips
    links['county_name'] = county_name
        
    use_cols = ['geometry', 'shstReferenceId', 'shstGeometryId',
                'fromIntersectionId', 'toIntersectionId', 'u', 'v',
                'roadClass','name', 'service', 'ref', 'lanes', 'maxspeed',
                'roadway','drive_access', 'walk_access', 'bike_access',
                'state_fips', 'county_fips', 'county_name'
               ]

    
    #if count == 0:
    #    links[use_cols].to_sql('links_step8', engine, if_exists='replace', index=False)
    else:
        links[use_cols].to_sql('links_step8', engine, if_exists='append', index=False)
    
    count = count + 1

SanJoaquin:True


C:\Anaconda3\envs\ranch\lib\site-packages\geopandas\geodataframe.py:1350: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


Create the geometry column

In [14]:
engine.execute("SELECT AddGeometryColumn('links_step8', 'geom', 4326, 'LINESTRING', 'XY');")
engine.execute("UPDATE links_step8 SET geom=GeomFromWKB(geometry, 4326) WHERE geom is null;")

Create indexes

In [15]:
engine.execute("CREATE INDEX ix_links_step8_fips ON links_step8(state_fips, county_fips);")
engine.execute("SELECT CreateSpatialIndex('links_step8', 'geom');")